In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math

In [3]:
df = pd.read_csv("data/us_estdata_expanded3.csv")
df

,Unnamed: 0,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,...,PINCP_ORIG_ADJ,HINCP_ORIG_ADJ,SMOCP_ORIG_ADJ,GRNTP_ORIG_ADJ,WAGP_ORIG_ADJ,PERNP_ORIG_ADJ,UNEMP_PCT_ORIG_ADJ,OWN_JOB_ORIG,TYPE_ORIG,CBSA_NAME_ORIG
0,0,P,2018HU0637969,3,1,1104,2,17,1013097,134,...,40000.0,72300.0,944.0,913.0,36800.0,38000.0,0.036364,13058,0,218.0
1,1,P,2018HU0808038,5,5,505,3,24,1013097,158,...,51000.0,96400.0,1478.0,1310.0,47000.0,49700.0,0.045407,0,0,341.0
2,2,P,2018HU1051188,9,1,7310,4,6,1013097,76,...,43000.0,94200.0,1998.0,1620.0,37000.0,40000.0,0.045469,46774,0,13.0
3,3,P,2018HU0461416,2,1,4110,1,36,1013097,60,...,40000.0,90200.0,1840.0,1670.0,38000.0,40000.0,0.051709,18982,0,367.0
4,4,P,2018GQ0016994,5,1,51167,3,51,1013097,61,...,45000.0,90050.0,1558.0,1417.0,41150.0,42000.0,0.025789,13230,1,477.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253085,253085,P,2018HU0435287,5,2,1400,3,24,1013097,44,...,40000.0,70700.0,1061.0,995.0,32800.0,35000.0,0.057456,2551,1,321.0
253086,253086,P,2018HU0652247,8,2,900,4,8,1013097,34,...,40000.0,68500.0,933.0,940.0,30000.0,35000.0,0.035656,6684,3,-1.0
253087,253087,P,2018HU0192642,8,1,824,4,8,1013097,58,...,50000.0,95100.0,1545.0,1410.0,45000.0,48000.0,0.029020,116203,3,-1.0
253088,253088,P,2018HU0750302,7,1,5915,3,48,1013097,61,...,37000.0,69420.0,1004.0,1039.0,32800.0,34700.0,0.040172,30030,0,89.0


In [4]:
df["ORIGIN"].value_counts()

603700     8376
2500390    3995
1703400    3374
400100     3201
4804600    2861
           ... 
5500900      57
4806900      56
800400       53
2202100      51
2300600      49
Name: ORIGIN, Length: 975, dtype: int64

In [5]:
df["MED"] = np.where(df["NAICS"] == "MED", 1, 0)
df["MFG"] = np.where(df["NAICS"] == "MFG", 1, 0)
df["RET"] = np.where(df["NAICS"] == "RET", 1, 0)
df["EDU"] = np.where(df["NAICS"] == "EDU", 1, 0)
df["ADM"] = np.where(df["NAICS"] == "ADM", 1, 0)
df["FOD"] = np.where(df["NAICS"] == "FOD", 1, 0)
df["PRF"] = np.where(df["NAICS"] == "PRF", 1, 0)
df["TRN"] = np.where(df["NAICS"] == "TRN", 1, 0)
df["SRV"] = np.where(df["NAICS"] == "SRV", 1, 0)
df["FIN"] = np.where(df["NAICS"] == "FIN", 1, 0)
df["WHL"] = np.where(df["NAICS"] == "WHL", 1, 0)
df["AGR"] = np.where(df["NAICS"] == "AGR", 1, 0)
df["PUB"] = np.where(df["NAICS"] == "PUB", 1, 0)
df["INF"] = np.where(df["NAICS"] == "INF", 1, 0)
df["ENT"] = np.where(df["NAICS"] == "ENT", 1, 0)
df["REL"] = np.where(df["NAICS"] == "REL", 1, 0)
df["UTL"] = np.where(df["NAICS"] == "UTL", 1, 0)
df["EXT"] = np.where(df["NAICS"] == "EXT", 1, 0)
df["MNG"] = np.where(df["NAICS"] == "MNG", 1, 0)
df["CON"] = np.where(df["NAICS"] == "CON", 1, 0)
df["NO_IND"] = np.where(df["NAICS"].isna(), 1, 0)

In [6]:
df["NO_IND"].sum()

69836

In [7]:
# clean up the database (Biogeme Database can only have numerical values)
df = df.select_dtypes(['number'])
df = df.fillna(0)

In [8]:
df["CHOSEN"].value_counts()

102500     335
5500100    323
5310200    321
5500700    312
1200500    291
          ... 
4203207     40
2701403     40
2601703     40
2701402     39
2701503     37
Name: CHOSEN, Length: 2336, dtype: int64

In [9]:
# defining the chosen alterantive for each person explicitly (0 to 35, corresponding to staying and moving to one of the many PUMAs)
df['CHOSEN_PUMA'] = df['CHOSEN']
df['CHOSEN'] = 0
for i in range(1, 201): 
    var = 'ALT' + str(i) + '_PUMA'
    df['CHOSEN'] = np.where(df[var]==df['CHOSEN_PUMA'], i, df['CHOSEN'])
df["CHOSEN"] = np.where(df["STAY"] == 1, 0, df["CHOSEN"])

In [10]:
df["CHOSEN"].value_counts()

0    220046
1     33044
Name: CHOSEN, dtype: int64

In [11]:
df["IN_COLLEGE"] = np.where((df["SCHG"] == 15) | (df["SCHG"] == 16), 1, 0)
df["IN_COLLEGE"]

0         0
1         0
2         0
3         0
4         0
         ..
253085    0
253086    1
253087    0
253088    0
253089    1
Name: IN_COLLEGE, Length: 253090, dtype: int64

In [12]:
df["CHOSEN"].value_counts()

0    220046
1     33044
Name: CHOSEN, dtype: int64

In [13]:
df["STAY"].value_counts()

1    220046
0     33044
Name: STAY, dtype: int64

In [14]:
df["AGE_18_34"] = np.where(df["AGEP"] <= 34, 1, 0)
df["AGE_35_64"] = np.where((df["AGEP"] >= 35) & (df["AGEP"] <= 64), 1, 0)
df["AGE_OVER_65"] = np.where((df["AGEP"] >= 65), 1, 0)
df["FOREIGN"] = np.where(df["NATIVITY"] == 2, 1, 0)

In [15]:
df["AGE_18_22"] = np.where(df["AGEP"] <= 22, 1, 0)
df["AGE_23_29"] = np.where((df["AGEP"] >= 23) & (df["AGEP"] <= 29), 1, 0)
df["AGE_30_39"] = np.where((df["AGEP"] >= 30) & (df["AGEP"] <= 39), 1, 0)
df["AGE_40_49"] = np.where((df["AGEP"] >= 40) & (df["AGEP"] <= 49), 1, 0)
df["AGE_50_64"] = np.where((df["AGEP"] >= 50) & (df["AGEP"] <= 64), 1, 0)

In [16]:
df["AGEP"].describe()

count    253090.000000
mean         49.978952
std          18.852487
min          18.000000
25%          34.000000
50%          51.000000
75%          64.000000
max          96.000000
Name: AGEP, dtype: float64

In [17]:
df["EDU_LESS_HIGH"] = np.where(df["SCHL"] <= 15, 1, 0)
df["EDU_HIGH"] = np.where((df["SCHL"] <= 20) & (df["SCHL"] >= 16), 1, 0) 
df["EDU_COLLEGE"] = np.where(df["SCHL"] >= 21, 1, 0)
df["EDU_NOCOLLEGE"] = np.where(df["EDU_COLLEGE"] == 0, 1, 0)

In [18]:
df["WOMAN_CHILD"] = np.where((df["PAOC"] >= 1) & (df["PAOC"] <= 3), 1, 0)
df["UNEMPLOYED"] = np.where(df["ESR"] == 3, 1, 0)

In [19]:
df["MALE"] = np.where(df["SEX"] == 1, 1, 0)
df["FEMALE"] = np.where(df["SEX"] == 0, 1, 0)

In [20]:
df["MARRIED"] = np.where(df["MAR"] == 1, 1, 0)

In [21]:
# df["child_old"] = np.where(df["child"] == df["REC_CHILD"], 0, df["child"])
# df["child_old"].value_counts()
df["child"].value_counts()

0    175342
1     77748
Name: child, dtype: int64

In [22]:
df["REC_NO_MAR"] = np.where((df["MARHD"] == 1) | (df["MARHW"] == 1), 1, 0)
df["REC_NO_MAR"].value_counts()

0    249541
1      3549
Name: REC_NO_MAR, dtype: int64

In [23]:
df["MARHM_new"] = np.where(df["MARHM"] == 2, 0, df["MARHM"])
df["MARHM_new"].value_counts()

0.0    248786
1.0      4304
Name: MARHM_new, dtype: int64

In [24]:
df["married_old"] = np.where((df["MARHM"] == df["MARRIED"]), 0, df["MARRIED"])
df["married_old"].value_counts()

1    131884
0    121206
Name: married_old, dtype: int64

In [25]:
df["MILITARY"] = np.where(df["MIL"] == 1, 1, 0)

In [26]:
# df["HH_COST"] = np.where(df["TEN"] == 3, df["GRNTP"], df["SMOCP"])

In [27]:
# up to debate still
df["AGR_EXT"] = np.where((df["AGR"] == 1) | (df["EXT"] == 1), 1, 0)
df["HIGH_ED"] = np.where((df["MED"] == 1) | (df["EDU"] == 1) | (df["PRF"] == 1) | (df["MED"] == 1) | (df["FIN"] == 1) | (df["INF"] == 1) | (df["MED"] == 1), 1, 0)
df["LICENSE"] = np.where((df["SRV"] == 1) | (df["REL"] == 1), 1, 0)
df["OTHER_JOB"] = np.where((df["AGR_EXT"] == 0) & (df["HIGH_ED"] == 0) & (df["LICENSE"] == 0) & (df["NO_IND"] == 0), 1, 0)

In [28]:
df["MICRO_adj_ORIG"] = np.where((df["TYPE_ORIG"] == 3) | (df["TYPE_ORIG"] == 2), 1, 0)
df["METRO"] = np.where(df["TYPE_ORIG"] == 1, 1, 0)
df["T34"] = np.where(df["TYPE_ORIG"] == 0, 1, 0)

In [29]:
for i in range(1, 201):
    key = "ALT{0}_".format(i)
    df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
    df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
    df[key + "T34"] = np.where(df[key + "TYPE"] == 0, 1, 0)

/tmp/ipykernel_11731/3036172732.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "MICRO"] = np.where((df[key + "TYPE"] == 2) | (df[key + "TYPE"] == 3), 1, 0)
/tmp/ipykernel_11731/3036172732.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key + "METRO"] = np.where(df[key + "TYPE"] == 1, 1, 0)
/tmp/ipykernel_11731/3036172732.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once 

In [30]:
df["UNEMPLOYED"] = np.where((df["ESR"] == 3) | (df["ESR"] == 6), 1, 0)
df["IN_LF"] = np.where(df["ESR"] == 6, 0, 1)

/tmp/ipykernel_11731/2921326413.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["IN_LF"] = np.where(df["ESR"] == 6, 0, 1)


In [31]:
df["WORK2_MAR"] = np.where(df["FES"] == 1, 1, 0)
df["WORK1_MAR"] = np.where((df["FES"] <= 4) & (df["FES"] >= 2), 1, 0)
df["OTHER_FAMILY"] = np.where((df["HHT"] == 2) | (df["HHT"] == 3), 1, 0)

/tmp/ipykernel_11731/3935644085.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["WORK2_MAR"] = np.where(df["FES"] == 1, 1, 0)
/tmp/ipykernel_11731/3935644085.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["WORK1_MAR"] = np.where((df["FES"] <= 4) & (df["FES"] >= 2), 1, 0)
/tmp/ipykernel_11731/3935644085.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

In [32]:
df.loc[df["CBSA_NAME_ORIG"] == -1, "CBSA_NAME_ORIG"] = -2

In [33]:
# df["INTERNAL"] = df["ORIGIN"] == df["CHOSEN"]

In [35]:
df["MIGSP_ORIG"] = df["ORIGIN"].astype(str).str.zfill(7).str[0:2].astype(int)

/tmp/ipykernel_11731/598267874.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["MIGSP_ORIG"] = df["ORIGIN"].astype(str).str.zfill(7).str[0:2].astype(int)


In [36]:
# making the Biogeme Database that is used for the model estimation
database = db.Database('us_data', df)

In [37]:
# The following statement allows you to use the names of the
# variable as Python variable. (in the utility functions)
globals().update(database.variables)

In [38]:
num_alternatives = 200

In [39]:
c_move = Beta("c_move", -0.1706511, None, None, 0)

In [42]:
# Staying Choice Parameters to be Estimated
c_stay_age_18_22 = Beta("c_stay_age_18_22", -0.4297984, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", -0.572291, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", -0.337318, None, None, 0)
c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0.1951023, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 0.6659279, None, None, 0)
c_stay_child = Beta("c_stay_child", -0.49784, None, None, 0)
c_stay = Beta("c_stay", 15.7542, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0.1775434, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0.00000437, None, None, 0)
c_stay_college = Beta("c_stay_college", 0.7150696, None, None, 0)
c_stay_rec_child = Beta("c_stay_rec_child", 0.05054558, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", -1.138993, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", -0.5003451, None, None, 0)
c_stay_mil = Beta("c_stay_mil", -0.79351627, None, None, 0)
c_stay_2work_mar = Beta("c_stay_2work_mar", 1.020378, None, None, 0)
c_stay_1work_mar = Beta("c_stay_1work_mar", 0.8788182, None, None, 0)
c_stay_otherfamily = Beta("c_stay_otherfamily", 0.66886, None, None, 0)
c_stay_ownjob = Beta("c_stay_ownjob", 0.8845035, None, None, 0)
c_stay_rentcost = Beta("c_stay_rentcost", -0.0009956133, None, None, 0)
c_stay_owncost = Beta("c_stay_owncost", 0.00040436, None, None, 0)
c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, 0, 0)
c_stay_hh_val = Beta("c_stay_hh_val", 0.000001735451, None, None, 0)
c_stay_income = Beta("c_stay_income", 0.0000009994866, 0, None, 0)

In [43]:
c_stay_T34 = Beta("c_stay_T34", 0.4236324, None, None, 0)
c_stay_metro = Beta("c_stay_metro", 0, None, None, 0)
c_stay_micro = Beta("c_stay_micro", -0.0694856, None, None, 0)
c_stay_edu_college = Beta("c_stay_edu_college", -0.2157945, None, None, 0)
c_stay_edu_high = Beta("c_stay_edu_high", 0, None, None, 0)
c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0.04565692, None, None, 0)

In [44]:
# # defining the staying utility function
# V0 = c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_40_49 * AGE_40_49 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_child * child + c_stay + c_stay_metro * METRO + c_stay_micro * MICRO_adj_ORIG + c_stay_college * IN_COLLEGE + c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR + c_stay_rec_child * REC_CHILD + c_stay_mil * MILITARY + c_stay_hh_val * HH_MED_VAL_ORIG + MARRIED * (c_stay_2work_mar * WORK2_MAR + c_stay_1work_mar * WORK1_MAR) + c_stay_otherfamily * OTHER_FAMILY + c_stay_ownjob * OWN_JOB_ORIG / TOT_JOBS_ORIG + (c_stay_rentcost * GRNTP_ORIG_ADJ + c_stay_owncost * SMOCP_ORIG_ADJ) / HINCP_ORIG_ADJ * 12 + c_stay_foreign * FOREIGN + c_stay_edu_college * EDU_COLLEGE + c_stay_edu_high * EDU_HIGH

In [45]:
# defining the staying utility function
V0 = c_stay + \
c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_40_49 * AGE_40_49 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 \
+ c_stay_child * child + c_stay_rec_child * REC_CHILD \
+ c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR \
+ MARRIED * (c_stay_2work_mar * WORK2_MAR + c_stay_1work_mar * WORK1_MAR) + c_stay_otherfamily * OTHER_FAMILY \
+ c_stay_college * IN_COLLEGE + c_stay_mil * MILITARY + c_stay_ownjob * OWN_JOB_ORIG / TOT_JOBS_ORIG \
+ c_stay_hh_val * HH_MED_VAL_ORIG + (c_stay_rentcost * GRNTP_ORIG_ADJ + c_stay_owncost * SMOCP_ORIG_ADJ) \
+ c_stay_foreign * FOREIGN \
+ c_stay_edu_college * EDU_COLLEGE + c_stay_edu_nohigh * EDU_LESS_HIGH \
+ IN_LF * (c_stay_income * PINCP_ORIG_ADJ + c_stay_unemp_rate * UNEMP_PCT_ORIG_ADJ) \
+ c_stay_T34 * T34 + c_stay_micro * MICRO_adj_ORIG

In [46]:
print(V0)

(((((((((((((((((((((((c_stay(15.7542) + (c_stay_age_18_22(-0.4297984) * AGE_18_22)) + (c_stay_age_23_29(-0.572291) * AGE_23_29)) + (c_stay_age_30_39(-0.337318) * AGE_30_39)) + (c_stay_age_40_49(0) * AGE_40_49)) + (c_stay_age_50_64(0.1951023) * AGE_50_64)) + (c_stay_age_65(0.6659279) * AGE_OVER_65)) + (c_stay_child(-0.49784) * child)) + (c_stay_rec_child(0.05054558) * REC_CHILD)) + (c_stay_rec_mar(-1.138993) * MARHM_new)) + (c_stay_rec_nomar(-0.5003451) * REC_NO_MAR)) + (MARRIED * ((c_stay_2work_mar(1.020378) * WORK2_MAR) + (c_stay_1work_mar(0.8788182) * WORK1_MAR)))) + (c_stay_otherfamily(0.66886) * OTHER_FAMILY)) + (c_stay_college(0.7150696) * IN_COLLEGE)) + (c_stay_mil(-0.79351627) * MILITARY)) + ((c_stay_ownjob(0.8845035) * OWN_JOB_ORIG) / TOT_JOBS_ORIG)) + (c_stay_hh_val(1.735451e-06) * HH_MED_VAL_ORIG)) + ((c_stay_rentcost(-0.0009956133) * GRNTP_ORIG_ADJ) + (c_stay_owncost(0.00040436) * SMOCP_ORIG_ADJ))) + (c_stay_foreign(0.1775434) * FOREIGN)) + (c_stay_edu_college(-0.2157945) *

In [47]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
c_destchoice_dist=Beta('c_destchoice_dist', -0.0000002684104, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', -0.3057492, None, None, 0)
c_destchoice_unemp_rate = Beta("c_destchoice_unemp_rate", -2.419098, None, None, 0)
c_destchoice_hhcost = Beta("c_destchoice_hhcost", -2.2776849, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0.00006571441, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 3.996802, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 1.517685, None, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 4.233292, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 0.7225565, None, None, 0)
c_destchoice_pincp = Beta("c_destchoice_pincp", 0, 0, None, 0)
c_destchoice_entscore_18_34 = Beta("c_destchoice_entscore_18_34", 0.5341137, None, None, 0)
c_destchoice_entscore_35_64 = Beta("c_destchoice_entscore_35_64", 2.761052, None, None, 0)
c_destchoice_entscore_65 = Beta("c_destchoice_entscore_65", 2.096477, None, None, 0)
c_destchoice_military = Beta("c_destchoice_military", 0.0002028093, None, None, 0)

In [48]:
c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0.7080241, None, None, 0)
c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0.8975212, None, None, 0)
c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0.2369336, None, None, 0)
c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 0.8342382, None, None, 0)
c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 1.012737, None, None, 0)
c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", --0.4684013, None, None, 0)
c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0.5166342, None, None, 0)
c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 1.416916, None, None, 0)

In [49]:
c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 5.945615, None, None, 0)
c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 1.918109, None, None, 0)
c_destchoice_license_job = Beta("c_destchoice_license_job", 5.042017, None, None, 0)
c_destchoice_other = Beta("c_destchoice_other", 2.561332, None, None, 0)

In [50]:
c_destchoice_samestate = Beta("c_destchoice_samestate", 2.228781, None, None, 0)
c_destchoice_birthstate = Beta("c_destchoice_birthstate", 0.3655848, None, None, 0)
c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", -0.05823393, None, None, 0)
c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", -0.00002960449, None, None, 0)

In [54]:
# defining the utility functions for each of the moving PUMA alternatives
# defined using the exec to parse a string to save space
# can also use a loop to print out the statements and then copy/paste them to run
# can also just write each one manually
for i in range(num_alternatives):
    num = i + 1
    initialization = "V{0} = c_move + log(ALT{0}_POP) + (CBSA_NAME_ORIG != ALT{0}_CBSA) * (c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1)) + (CBSA_NAME_ORIG == ALT{0}_CBSA) * c_destchoice_cbsa_dist * ALT{0}_DIST + \
        \
    c_destchoice_hhcost * ALT{0}_HH_COST + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + \
        \
    c_destchoice_age_18_34 * AGE_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * AGE_35_64 * ALT{0}_35_64 / ALT{0}_POP + c_destchoice_age_over_65 * AGE_OVER_65 * ALT{0}_65 / ALT{0}_POP + \
        \
    ALT{0}_ENT / ALT{0}_EMP * (c_destchoice_entscore_18_34 * AGE_18_34 + c_destchoice_entscore_35_64 * AGE_35_64 + c_destchoice_entscore_65 * AGE_OVER_65) + \
        \
    IN_LF * (c_destchoice_unemp_rate * ALT{0}_UNEMP_PCT + c_destchoice_pincp * ALT{0}_PINCP) + \
        \
    (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * ALT{0}_T34 + \
    (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * ALT{0}_METRO + \
    (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * ALT{0}_MICRO + \
        \
    (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER_JOB) * ALT{0}_OWN_JOB / ALT{0}_EMP + c_destchoice_military * MILITARY * ALT{0}_MIL_NUM + \
        \
    c_destchoice_birthstate * (POBP == ALT{0}_State) + c_destchoice_samestate * (MIGSP_ORIG == ALT{0}_State)".format(num)
    exec(initialization)
print(V5)

# full model specification (takes a bit longer to run):
# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_destchoice_hhcost * ALT{0}_HH_COST + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + c_destchoice_age_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * ALT{0}_35_64 / ALT{0}_POP + c_destchoice_age_over_65 * ALT{0}_65 / ALT{0}_POP + c_destchoice_entscore * ALT{0}_ENT / AGEP / ALT{0}_EMP + c_destchoice_unemp * ALT{0}_UNEMP + c_destchoice_pincp * ALT{0}_PINCP + \
# (c_destchoice_T34_T34 * T34 + c_destchoice_Metro_T34 * METRO +  c_destchoice_Micro_T34 * MICRO_adj_ORIG) * ALT{0}_T34 + \
# (c_destchoice_T34_Metro * T34 + c_destchoice_Metro_Metro * METRO + c_destchoice_Micro_Metro * MICRO_adj_ORIG) * ALT{0}_METRO + \
# (c_destchoice_T34_Micro * T34  + c_destchoice_Metro_Micro * METRO + c_destchoice_Micro_Micro * MICRO_adj_ORIG) * ALT{0}_MICRO + \
# (c_destchoice_geo_spec_job * AGR_EXT + c_destchoice_high_ed_job * HIGH_ED + c_destchoice_license_job * LICENSE + c_destchoice_other * OTHER) * ALT{0}_OWN_JOB / ALT{0}_EMP".format(num)

# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_adm * ADM * ALT{0}_ADM/ALT{0}_EMP + c_agr * AGR * ALT{0}_AGR/ALT{0}_EMP + c_con * CON * ALT{0}_CON/ALT{0}_EMP + c_edu * EDU * ALT{0}_EDU/ALT{0}_EMP + c_ent * ENT * ALT{0}_ENT/ALT{0}_EMP + c_ext * EXT * ALT{0}_EXT/ALT{0}_EMP + c_fin * FIN * ALT{0}_FIN/ALT{0}_EMP + c_fod * FOD * ALT{0}_FOD/ALT{0}_EMP + c_inf * INF * ALT{0}_INF/ALT{0}_EMP + c_med * MED * ALT{0}_MED/ALT{0}_EMP + c_mfg * MFG * ALT{0}_MFG/ALT{0}_EMP + c_mng * MNG * ALT{0}_MNG/ALT{0}_EMP + c_prf * PRF * ALT{0}_PRF/ALT{0}_EMP + c_pub * PUB * ALT{0}_PUB/ALT{0}_EMP + c_rel * REL * ALT{0}_REL/ALT{0}_EMP + c_ret * RET * ALT{0}_RET/ALT{0}_EMP + c_srv * SRV * ALT{0}_SRV/ALT{0}_EMP + c_trn * TRN * ALT{0}_TRN/ALT{0}_EMP + c_utl * UTL * ALT{0}_UTL/ALT{0}_EMP + c_whl * WHL * ALT{0}_WHL/ALT{0}_EMP"

# for the fields already in the Biogeme db.Database, can explicitly refer to them; also used a few references to other databases using .loc and fields in the Biogeme database

((((((((((((((((((log(ALT5_POP) + ((CBSA_NAME_ORIG != ALT5_CBSA) * ((c_destchoice_dist(-2.684104e-07) * ALT5_DIST) + (c_destchoice_logdist(-0.3057492) * log((ALT5_DIST + `1`)))))) + (((CBSA_NAME_ORIG == ALT5_CBSA) * c_destchoice_cbsa_dist(-2.960449e-05)) * ALT5_DIST)) + (c_destchoice_hhcost(-2.2776849) * ALT5_HH_COST)) + ((c_destchoice_college(6.571441e-05) * IN_COLLEGE) * ALT5_COLLEGE)) + (((c_destchoice_foreign(0.7225565) * FOREIGN) * ALT5_FOREIGN) / ALT5_POP)) + (((c_destchoice_age_18_34(3.996802) * AGE_18_34) * ALT5_18_34) / ALT5_POP)) + (((c_destchoice_age_35_64(1.517685) * AGE_35_64) * ALT5_35_64) / ALT5_POP)) + (((c_destchoice_age_over_65(4.233292) * AGE_OVER_65) * ALT5_65) / ALT5_POP)) + ((ALT5_ENT / ALT5_EMP) * (((c_destchoice_entscore_18_34(0.5341137) * AGE_18_34) + (c_destchoice_entscore_35_64(2.761052) * AGE_35_64)) + (c_destchoice_entscore_65(2.096477) * AGE_OVER_65)))) + (IN_LF * ((c_destchoice_unemp_rate(-2.419098) * ALT5_UNEMP_PCT) + (c_destchoice_pincp(0) * ALT5_PINCP)

In [55]:
utilities = {}
for i in range(num_alternatives + 1):
    init = "utilities[{0}] = V{0}".format(i)
    exec(init)
V = utilities

In [56]:
av = {}
for i in range(num_alternatives + 1):
    av[i] = 1

In [57]:
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDetailed()
logger.allMessages()

'*** Messages from biogeme 3.2.8 [2022-11-29]\n'

In [58]:
move = Beta("move", 1.497034, 1, None, 0)

In [59]:
# nest_move = move, list(range(1, num_alternatives + 1))
# nest_stay = 1.0, [0]
# nests = nest_move, nest_stay

In [60]:
# nest_logprob = models.lognested(V, av, nests, CHOSEN)

# biogeme_nest = bio.BIOGEME(database, nest_logprob, suggestScales=False)
# biogeme_nest.modelName = "nested_full_full"

# biogeme_nest.calculateNullLoglikelihood(av)

# results_nest = biogeme_nest.estimate()
# pandasResults_nest = results_nest.getEstimatedParameters()
# print(pandasResults_nest)

In [61]:
# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
# estimating the CHOSEN field
logprob = models.loglogit(V, av, CHOSEN)

# formulas = {"loglike": logprob, "weight": W0}

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob, suggestScales=False)
biogeme.modelName = 'us_full_final'

biogeme.saveIterations = False

[20:02:31] < General >   Remove 1046 unused variables from the database as only 3840 are used.


In [61]:
# Calculate the null log likelihood for reporting. (likelihood of predicting every entry's alterantive correctly if alternatives are randomly chosen)
biogeme.calculateNullLoglikelihood(av)

-671106.7195903357

In [ ]:
#estimate parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

In [ ]:
# # occupation-based coefficients

# c_adm = Beta("c_adm", 0, None, None, 0)
# c_agr = Beta("c_agr", 0, None, None, 0)
# c_con = Beta("c_con", 0, None, None, 0)
# c_edu = Beta("c_edu", 0, None, None, 0)
# c_ent = Beta("c_ent", 0, None, None, 0)
# c_ext = Beta("c_ext", 0, None, None, 0)
# c_fin = Beta("c_fin", 0, None, None, 0)
# c_fod = Beta("c_fod", 0, None, None, 0)
# c_inf = Beta("c_inf", 0, None, None, 0)
# c_med = Beta("c_med", 0, None, None, 0)
# c_mfg = Beta("c_mfg", 0, None, None, 0)
# c_mng = Beta("c_mng", 0, None, None, 0)
# c_prf = Beta("c_prf", 0, None, None, 0)
# c_pub = Beta("c_pub", 0, None, None, 0)
# c_rel = Beta("c_rel", 0, None, None, 0)
# c_ret = Beta("c_ret", 0, None, None, 0)
# c_srv = Beta("c_srv", 0, None, None, 0)
# c_trn = Beta("c_trn", 0, None, None, 0)
# c_utl = Beta("c_utl", 0, None, None, 0)
# c_whl = Beta("c_whl", 0, None, None, 0)

In [ ]:
# # Staying Choice Parameters with starting values of 0
# c_stay_married = Beta("c_stay_married", 0, None, None, 0)
# c_stay_age_18_22 = Beta("c_stay_age_18_22", 0, None, None, 0)
# c_stay_age_23_29 = Beta("c_stay_age_23_29", 0, None, None, 0)
# c_stay_age_30_39 = Beta("c_stay_age_30_39", 0, None, None, 0)
# c_stay_age_40_49 = Beta("c_stay_age_40_49", 0, None, None, 1)
# c_stay_age_50_64 = Beta("c_stay_age_50_64", 0, None, None, 0)
# c_stay_age_65 = Beta("c_stay_age_65", 0, None, None, 0)
# c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0, None, None, 0)
# c_stay_edu_somecollege = Beta("c_stay_edu_somecollege", 0, None, None, 0)
# c_stay_edu_college = Beta("c_stay_edu_college", 0, None, None, 0)
# c_stay_edu_beyondhs = Beta('c_stay_edu_beyondhs', 0, None, None, 0)
# c_stay_child = Beta("c_stay_child", 0, None, None, 0)
# c_stay_unemployed = Beta("c_stay_unemployed", 0, None, None, 0)
# c_stay = Beta("c_stay", 15, None, None, 0)
# c_stay_foreign = Beta("c_stay_foreign", 0, None, None, 0)
# c_stay_dens = Beta("c_stay_dens", 0, None, None, 0)
# c_stay_college = Beta("c_stay_college", 0, None, None, 0)
# c_stay_rec_child = Beta("c_stay_rec_child", 0, None, None, 0)
# c_stay_rec_mar = Beta("c_stay_rec_mar", 0, None, None, 0)
# c_stay_rec_nomar = Beta("c_stay_rec_nomar", 0, None, None, 0)
# c_stay_mil = Beta("c_stay_mil", 0, None, None, 0)
# c_stay_2work_mar = Beta("c_stay_2work_mar", 0, None, None, 0)
# c_stay_1work_mar = Beta("c_stay_1work_mar", 0, None, None, 0)
# c_stay_otherfamily = Beta("c_stay_otherfamily", 0, None, None, 0)
# c_stay_income = Beta("c_stay_income", 0, None, None, 0)
# c_stay_hhinc = Beta("c_stay_hhinc", 0, None, None, 0)
# c_stay_hurent = Beta('c_stay_hurent', 0, None, None, 0)
# c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
# c_stay_hhcost = Beta("c_stay_hhcost", 0, None, None, 0)
# c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, None, 0)
# c_stay_hh_val = Beta("c_stay_hh_val", 0, None, None, 0)
# c_stay_partcp = Beta("c_stay_partcp", 0, None, None, 0)
# c_stay_rentcost = Beta("c_stay_rentcost", 0, None, None, 0)
# c_stay_owncost = Beta("c_stay_owncost", 0, None, None, 0)

In [ ]:
# # Bounded staying choice parameters
# c_stay_income = Beta("c_stay_income", 0, 0, None, 0)
# c_stay_hhinc = Beta("c_stay_hhinc", 0, 0, None, 0)
# c_stay_hurent = Beta('c_stay_hurent', 0, None, 0, 0)
# c_stay_vac = Beta('c_stay_vac', 0, None, None, 0)
# c_stay_hhcost = Beta("c_stay_hhcost", 0, None, 0, 0)
# c_stay_unemp_rate = Beta("c_stay_unemp_rate", 0, None, 0, 0)
# c_stay_hh_val = Beta("c_stay_hh_val", 0, 0, None, 0)

In [ ]:
# # Destination Choice Parameters to be estimated
# # Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not
# c_destchoice_dist=Beta('c_destchoice_dist', 0, None, None, 0)
# c_destchoice_logdist=Beta('c_destchoice_logdist', 0, None, None, 0)
# c_destchoice_unemp_rate = Beta("c_destchoice_unemp", 0, None, None, 0)
# c_destchoice_hhcost = Beta("c_destchoice_hhcost", 0, None, None, 0)
# c_destchoice_college = Beta("c_destchoice_college", 0, None, None, 0)
# c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 0, None, None, 0)
# c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 0, None, None, 0)
# c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 0, None, None, 0)
# c_destchoice_foreign = Beta("c_destchoice_foreign", 0, None, None, 0)
# c_destchoice_pincp = Beta("c_destchoice_pincp", 0, None, None, 0)

# c_destchoice_entscore_18_34 = Beta("c_destchoice_entscore_18_34", 0, None, None, 0)
# c_destchoice_entscore_35_64 = Beta("c_destchoice_entscore_35_64", 0, None, None, 0)
# c_destchoice_entscore_65 = Beta("c_destchoice_entscore_65", 0, None, None, 0)

# c_destchoice_T34_T34 = Beta("c_destchoice_T34_T34", 0, None, None, 1) 
# c_destchoice_T34_Metro = Beta('c_destchoice_T34_Metro', 0, None, None, 0)
# c_destchoice_T34_Micro = Beta("c_destchoice_T34_Micro", 0, None, None, 0)
# c_destchoice_Metro_T34 = Beta("c_destchoice_Metro_T34", 0, None, None, 0)
# c_destchoice_Metro_Metro = Beta('c_destchoice_Metro_Metro', 0, None, None, 0)
# c_destchoice_Metro_Micro = Beta('c_destchoice_Metro_Micro', 0, None, None, 0)
# c_destchoice_Micro_T34 = Beta("c_destchoice_Micro_T34", 0, None, None, 0)
# c_destchoice_Micro_Metro = Beta("c_destchoice_Micro_Metro", 0, None, None, 0)
# c_destchoice_Micro_Micro = Beta("c_destchoice_Micro_Micro", 0, None, None, 0)

# c_destchoice_geo_spec_job = Beta("c_destchoice_geo_spec_job", 0, None, None, 0)
# c_destchoice_high_ed_job = Beta("c_destchoice_high_ed_job", 0, None, None, 0)
# c_destchoice_license_job = Beta("c_destchoice_license_job", 0, None, None, 0)
# c_destchoice_other = Beta("c_destchoice_other", 0, None, None, 0)

# c_destchoice_samestate = Beta("c_destchoice_samestate", 0, None, None, 0)
# c_destchoice_birthstate = Beta("c_destchoice_birthstate", 0, None, None, 0)
# c_destchoice_samecbsa = Beta("c_destchoice_samecbsa", 0, None, None, 0)
# c_destchoice_cbsa_dist = Beta("c_destchoice_cbsa_dist", 0, None, None, 0)
# c_destchoice_military = Beta("c_destchoice_military", 0, None, None, 0)